# Testing the file workings

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv(override=True)

print("OpenAI Key:", os.getenv("OPENAI_API_KEY") is not None)
print("Claude Key:", os.getenv("ANTHROPIC_API_KEY") is not None)
print("Gemini Key:", os.getenv("GOOGLE_API_KEY") is not None)

In [ ]:
import os
os.environ.pop("ANTHROPIC_API_KEY", None)

print("After removal:", repr(os.environ.get("ANTHROPIC_API_KEY")))

## API .env calls

In [ ]:
os.getenv("OPENAI_API_KEY")

In [ ]:
os.getenv("ANTHROPIC_API_KEY")

In [ ]:
os.getenv("GOOGLE_API_KEY")

## Basic LLM Calls

In [ ]:
import os
import asyncio

from models.openai_model import call_openai
from models.claude_model import call_claude
from models.gemini_model import call_gemini


OPENAI_KEY = os.getenv("OPENAI_API_KEY")
CLAUDE_KEY = os.getenv("ANTHROPIC_API_KEY")
GEMINI_KEY = os.getenv("GOOGLE_API_KEY")


async def test_all_models():

    prompt = 'Return ONLY valid JSON: {"status": "working"}'

    print("Testing OpenAI...")
    openai_result = await call_openai(prompt, api_key=OPENAI_KEY)
    print("OpenAI Result:", openai_result)

    print("\nTesting Claude...")
    claude_result = await call_claude(prompt, api_key=CLAUDE_KEY)
    print("Claude Result:", claude_result)

    print("\nTesting Gemini...")
    gemini_result = await call_gemini(prompt, api_key=GEMINI_KEY)
    print("Gemini Result:", gemini_result)

    return {
        "openai": openai_result,
        "claude": claude_result,
        "gemini": gemini_result
    }


results = await test_all_models()


In [ ]:
async def test_parallel():

    prompt = 'Return ONLY valid JSON: {"status": "parallel_working"}'

    tasks = [
        call_openai(prompt, api_key=OPENAI_KEY),
        call_claude(prompt, api_key=CLAUDE_KEY),
        call_gemini(prompt, api_key=GEMINI_KEY)
    ]

    results = await asyncio.gather(*tasks)

    print("OpenAI:", results[0])
    print("Claude:", results[1])
    print("Gemini:", results[2])

await test_parallel()


## Testing Segmentation

In [ ]:
sample_contract="""
1. PAYMENT TERMS
Client shall pay all undisputed invoices within thirty (30) days of receipt. Late payments shall incur interest at 2% per month.

2. INDEMNIFICATION
Vendor shall indemnify, defend, and hold harmless Client against any and all third-party claims arising from Vendor’s breach of this Agreement.

3. LIMITATION OF LIABILITY
In no event shall Vendor’s total liability exceed the total fees paid under this Agreement during the preceding twelve (12) months.

4. DATA SECURITY
Vendor shall implement commercially reasonable administrative, technical, and physical safeguards to protect Client Data from unauthorized access.

5. TERMINATION FOR CONVENIENCE
Client may terminate this Agreement at any time without cause upon thirty (30) days written notice.

6. CONFIDENTIALITY
Each party agrees to maintain the confidentiality of all non-public information disclosed under this Agreement.

7. FORCE MAJEURE
Neither party shall be liable for failure to perform due to events beyond reasonable control including natural disasters or governmental actions.

8. GOVERNING LAW
This Agreement shall be governed by and construed in accordance with the laws of the State of New York.

9. AUTO-RENEWAL
This Agreement shall automatically renew for successive one-year terms unless either party provides written notice at least sixty (60) days prior to expiration.

10. ASSIGNMENT
Vendor may not assign this Agreement without prior written consent of Client.


"""

In [ ]:
from core.segmentation import segment_contract
clauses = await segment_contract(sample_contract)

print("Segmentation Output:")
print(clauses)

In [ ]:
from core.analysis import initial_analysis
from core.arbitration import arbitration
from core.review import review_round

clause_text = clauses[0]["clause_text"]

initial = await initial_analysis(clause_text)
reviewed = await review_round(clause_text, initial)

final = await arbitration(clause_text, reviewed)

print("Final Arbitration Output:")
print(final)


In [ ]:
import time

async def test_multiple_clauses(clauses_subset):

    results = []

    start = time.time()

    for i, clause in enumerate(clauses_subset):
        print(f"\nProcessing clause {i+1}/{len(clauses_subset)}")

        initial = await initial_analysis(clause["clause_text"])
        reviewed = await review_round(clause["clause_text"], initial)
        final = await arbitration(clause["clause_text"], reviewed)

        results.append(final)

    end = time.time()

    print("\nTotal Time:", round(end - start, 2), "seconds")
    print("Average per clause:", round((end - start)/len(clauses_subset), 2), "seconds")

    return results


In [ ]:
results = await test_multiple_clauses(clauses[:5])


In [ ]:
anonymized = {
    "Response A": {
        "golden_clause_detected": True,
        "golden_clause_type": "Payment",
        "risk_score": 6.0,
        "balanced": False,
        "justification": "Interest rate high.",
        "key_risk_indicators": ["2% per month"]
    },
    "Response B": {
        "golden_clause_detected": True,
        "golden_clause_type": "Payment",
        "risk_score": 5.0,
        "balanced": True,
        "justification": "Standard clause.",
        "key_risk_indicators": []
    },
    "Response C": {
        "golden_clause_detected": False,
        "golden_clause_type": None,
        "risk_score": 2.0,
        "balanced": True,
        "justification": "Not risky.",
        "key_risk_indicators": []
    }
}


In [ ]:
import json
from config.prompts import REVIEW_PROMPT

responses_text = ""
for label, content in anonymized.items():
    responses_text += f"{label}:\n{json.dumps(content, indent=2)}\n\n"

prompt = REVIEW_PROMPT.format(
    clause_text="Client shall pay within 30 days. Late payment incurs 2% monthly interest.",
    responses_text=responses_text
)


In [ ]:
print(prompt)

In [ ]:
from models.openai_model import call_openai
from models.claude_model import call_claude
from models.gemini_model import call_gemini

raw = await call_openai(prompt, api_key=os.getenv("OPENAI_API_KEY"))
print(raw)


In [ ]:
from core.schemas import SingleReviewOutput

validated = SingleReviewOutput(**raw)
print(validated)


In [ ]:
validated

In [ ]:
initial = await initial_analysis(clause_text)
reviewed = await review_round(clause_text, initial)

print(reviewed)

In [ ]:
# ===== FULL REVIEW PIPELINE TEST =====

import os
import json
import asyncio

from core.analysis import initial_analysis
from core.review import review_round
from core.schemas import SingleReviewOutput

# Sample clause
clause_text = "Client shall pay all undisputed invoices within thirty (30) days of receipt. Late payments shall incur interest at 2% per month."

print("Running Initial Analysis...")
initial = await initial_analysis(clause_text)

print("\nInitial Analysis Output:")
print(json.dumps(initial, indent=2))


print("\nRunning Review Round...")
reviewed = await review_round(clause_text, initial)

print("\nReview Round Output Structure:")
print(json.dumps(reviewed, indent=2))


In [ ]:
reviewed

In [ ]:
# ===== STRUCTURE VALIDATION CHECK =====

print("\nValidating structure...")

assert "responses" in reviewed
assert "reviews" in reviewed

for reviewer_name, review_data in reviewed["reviews"].items():
    validated = SingleReviewOutput(**review_data)
    
    # Ensure evaluation contains A, B, C
    assert set(validated.evaluation.keys()) == {"Response A", "Response B", "Response C"}
    
    # Ensure ranking contains 1, 2, 3
    assert set(validated.ranking.keys()) == {"1", "2", "3"}

print("✅ Review stage structure validated successfully.")


In [ ]:
from core.analysis import initial_analysis
from core.review import review_round
from core.arbitration import arbitration
import json

clause_text = "Client shall pay all undisputed invoices within thirty (30) days of receipt. Late payments shall incur interest at 2% per month."

initial = await initial_analysis(clause_text)
reviewed = await review_round(clause_text, initial)

final = await arbitration(clause_text, reviewed)

print("Final Arbitration Output:")
print(json.dumps(final, indent=2))

In [ ]:
reviewed

## Testing the segmentation

In [ ]:
contract_text = """
Client shall pay all undisputed invoices within thirty (30) days of receipt.
Late payments shall incur interest at 2% per month.

Vendor shall indemnify, defend, and hold harmless Client against any and all third-party claims arising from Vendor’s breach of this Agreement.

The parties agree to cooperate in good faith to resolve disputes amicably.
"""


In [ ]:
sample_contract_1="""

1.0 Service Level Agreement Overview 
This document is a Service Level Agreement between Northwestern IT (service provider) 
and the  
(customer), outlining the provisioning of IT services. 
The purpose of this agreement is to establish service expectations, identify provider and 
consumer responsibilities related to this service, specify how this service is supported, and 
clarify service charges. 
2.0 Business Service Description 
Northwestern IT (NUIT) agrees to provide Endpoint device management and support as a 
service to customer. 
NUIT will: provide 
• Device deployment – Procurement, Custom factory image loading including 
enterprise licensed common applications, asset tagging and labelling, pre-delivery 
and staging services, and system installation, system configuration and data 
migrations. Delivers and installs your fully integrated systems to customer 
designated user locations. 
• Secure endpoint management - pre-configuration of security solutions and 
installation of antivirus, data encryption and data backup solutions. Regular 
application of Operating system and application software patches. 
• Device Management - Agent monitoring to ensure proper tracking of device 
lifecycle 
• NUIT Service Desk - represents the entry point and first-touch support process for 
all hardware and software service-related requests. NUIT delivers a strategic, 
customized full-featured service desk that is driven by the goals of quality, value, 
and customer satisfaction. NUIT service desk is accountable to resolve, track, 
escalate and follow-up on all Endpoint systems related issues that affect customer 
productivity. 
• NUIT provides on-site Field Services support for endpoint systems, software, and 
peripherals support. We assist with installations, moves, adds and changes, as well 
as everyday hardware and peripheral support services. 
Authorized service delivery providers will be dispatched to your End User when 
hardware, software, or peripheral problems cannot be resolved remotely. 
• Disposition and redeployment– NUIT delivers efficient, secure and simple 
processes for data recoveries to disposals and refreshes. Ensures that system data 
is destroyed for security purposes. Disposal of Assets and manages Employee 
Purchase Programs. 
3.0 Agreement Period 
The period of this agreement is September 1, 2019 through August 30, 2020. Each year, 
this agreement will automatically extend for one calendar year unless it is modified or 
terminated as described in this document. 
4.0 Maintenance 
Maintenance of services provided in this agreement will be conducted in accordance with 
established NUIT Change Management practices, and will occur during scheduled service 
maintenance windows. Emergency maintenance, in response to service outages, may 
occur with no advanced notice to service consumers. Maintenance or outages of other, 
underlying or integrated, services may impact the services provided in this agreement. 
Notification of service maintenance and outages is available at: 
Service Status (https://service-status.northwestern.edu/). 
Information on how to subscribe to email service alerts can be found at: 
https://service-status.northwestern.edu/ 
5.0 Northwestern IT Responsibilities 
The following list specifies responsibilities of NUIT related to providing the service set 
forth in this agreement. Northwestern IT will be responsible for the following: 
Workstations (desktop and laptop computers) 
• Build workstations 
o Install one of NUIT approved computer images 
 Dual booting or machines with multiple operating systems will be 
assessed and approved by the Technology Support Services Director on 
a case by case basis based on business, research or teaching need. 
o Install additional software and or make additional updates to the endpoint 
device 
 Upon request 
 Subject to security approval 
o Include and manage 
 NUIT’ service automated software updating services (Kace/ JAMF) 
 Endpoint security software(Symantec Endpoint Protection) 
 Whole disk encryption (MBAM and FileVault) 
o Ensure desired work space has functional electrical and network access 
 Make recommendations for electrical or network access and advise 
CONDUITS and FM ordering processes. 
o Purchase, deliver and install workstation in desired work space 
o Troubleshooting wired or wireless network connectivity users, escalating to 
TNS has required. 
• Address user observed errors upon notification 
o Conduct troubleshooting and remediation efforts 
o Coordinate warranty work as required 
 Coordinate with Dell, Lenovo/CDWG for services (PCs) 
 Coordinate with Apple service technician (MACs) 
o Provide temporary loaner computer 
 Upon request 
 If operationally necessary 
 Subject to availability 
o Conference room and classroom support 
 Coordinate with Northwestern IT Audio/Visual team as need for 
support issue 
• Provide departmental fiscal office with semi-annual refresh notifications for 
workstations. 
• Assist with providing purchasing recommendations for new or replacement computers 
• Provide major OS updates as required for vendor support (e.g. Windows “Creator 
Updates”, MacOS 10.X point releases) 
• Provide disposition of end-of-life devices 
o Scrub data and remove hard drive 
o Send devices to Northwestern University Surplus and/or donate to Student 
Enrichment Services 
Mobile Devices 
The following support is offered upon request: 
• Make purchasing recommendations for Northwestern University owned mobile device 
• Configure mobile devise 
• Configure email 
• Coordinate activities with 3rd party vendors as required (University owned devices 
only) 
• Address user observed errors upon notification 
o Conduct troubleshooting and remediation efforts 
o Coordinate warranty work as required (University owned devices only) 
Printers 
The following support is offered upon request: 
• Set up printers 
• Organize for disposal of printers 
OU Administration (CampusAD) 
The following support is offered upon request: 
• Add new users to CampusAD 
• Create CampusAD accounts 
• Grant access to resources control via CampusAD 
• Delete users 
• Manage Group Policies 
Endpoint Management Services 
Management of the following endpoint management software’s: 
• Workstations will be patched no less than monthly for OS and application by use of 
management software, including Kace, JAMF 
• Full system backup of device including local machine stored data using Crashplan 
o Monitoring of backup services will be provided, with either tickets generated 
or user notification provided in the event that backups are not functioning 
Box 
The following support is offered upon request: 
• Install Box client software, such as Box Edit, Drive and Sync 
• Facilitate the creation of organization shares and group accounts 
Multi-function devices 
The following support is offered upon request: 
• Configure printing and scanning access to a multi-function device. 
• Address user observed errors upon notification 
o Conduct troubleshooting and remediation efforts 
o Coordinate warranty work as required 
Installation and Troubleshooting of customer specific software 
The following support is offered upon request: 
• Northwestern IT will install all customer specific software and will work with the 
user/experts in said software to ensure that the installation of these applications does not 
destabilize other desktop or application functionality 
• NUIT will work with the user/experts to contact the vendor of said software to 
accomplish same, but will not act independently of the user/experts in contacting 
vendors. 
Out of Scope 
The following services are explicitly not offered in this SLA. 
• Management of personal devices 
• Support for unmanaged devices such as servers, customer purchased devices 
New Technology 
Both Northwestern IT and the customer recognize that new technology adoption is 
important to the mission of the university. The customer may be introduced or considered 
for adoption during the course of this Agreement. At the mutual agreement of the NUIT 
and the customer, joint testing, demonstration, or evaluation of such technology may be 
undertaken. 
If the customer adopts a new computing technology and desires that NUIT support it, then 
the Customer Contact will consult with the TSS.  If they agree that the new technology 
has a limited effect on support time for the support personnel, then it will be incorporated 
into the current support activities. Otherwise, the Customer Contact will follow the 
procedure described in the section entitled “Support Outside of this Agreement”. 
Computer Support Policies 
Northwestern IT will support the University-owned computers of the customer based on 
the following policies: 
1. New purchases must meet the Northwestern IT recommended standards. New 
purchases will be set up with the NUIT software image. 
2. Existing computer hardware must meet the minimum hardware and operating system 
standards. 
3. Support level will be determined by the age of the machine 
Computer Age 
Support level 
Support description 
0-4 years 
Full Support 
Northwestern IT will ensure the working order of the 
computer, using reasonable staff time and support 
resources 
4-5 years 
Best Effort 
Northwestern IT will spend up to one hour fixing an 
issue. If the issue cannot be resolved at that time, NUIT 
will recommend replacing the computer and facilitate the 
recovery/transfer of data. 
5 years or 
older 
Out of Support 
Northwestern IT will facilitate the recovery/transfer of 
data, but will not attempt to resolve technical issues. 
Legacy Computers connected to instruments 
• Best effort support will be provided to computers connected to research equipment in 
consultation with Northwestern IT’s Information Security Office 
• Legacy equipment may need to be placed off network to avoid remote exploit or other 
risk 
6.0 Customer Responsibilities 
The following list contains the customer’s responsibilities related to their use of this 
service: 
General 
• Submit requests for services to the IT Support Center (by email, phone, or self-service 
ticket entry) 
• Submit notification of service interruptions or reductions to the IT Support Center 
• Provide funding for the purchase of any equipment covered by this SLA 
• Provide a list of the devices and their serial numbers for each device covered by this 
SLA: 
o Initial list will be provided from Kace export 
o Updates and changes to support environment will be managed in Kace, or its 
replacement for asset management 
• Notify NUIT of devices to be added to the SLA throughout the year through the new 
computer request form 
Workstations 
• Arrange to refresh workstations within four years of purchase 
• Provide funding for the refresh purchase 
Printers 
• Purchase a service warranty for each University owned printer that includes break-fix 
and full replacement 
7.0 Assumptions 
This service agreement assumes the following: 
• NUIT will bill Customer for support activity outside of the scope of this agreement at 
a standard service rate 
• Overtime work will be charged at 150% of the standard rate 
• After hours support work that requires support personal to return to work will include 
a minimum charge of three hours 
8.0 Requesting Support 
Support tickets, pertaining to service incidents (break-fix) and requests, should be 
submitted via the NUIT Service Desk and include the following information: 
• Customer contact information including: name, phone number, email address 
• Brief description of the service issue 
• Priority level of the incident or request 
The Service Desk staff will provide first level support to resolve the issue. Issues that are 
not resolved at first level will be escalated to Tier 2 support teams for resolution. 
Telephone calls received by the Service Desk outside of scheduled support hours will 
prompt callers to leave a message or page support staff for emergencies. The voice 
messages will be responded to on next business day. 
Customers are encouraged to check the NUIT Service Status web site 
https://service-status.northwestern.edu/ for scheduled maintenance events and service 
alerts. 
Contact Information 
Days 
Hours 
of 
operati
on 
Monday – 
Friday 
8:00 AM – 6:00 PM 
Service 
Desk 
Phone: (847) 491-4357 
(1-HELP) 
https://www.it.northwe 
stern.edu/supportcenter 
/index.html 
Saturday 
Not Available 
Sunday 
Summer Hours Monday – Friday 8:00 am – 5:00pm 
Not Available 
CLOSED: Winter/Spring Break, University holidays 
8.1 Incident Response and Resolution Times 
All incident tickets are assessed for customer impact and urgency to determine priority. A 
predefined Incident priority matrix is used as guidance in assessing incident priority. 
Priority 
Target 
Response 
Time 
Target 
Resolution 
Time 
Critical 
(P1) 
Immediate 
2 Hour 
High 
(P2) 
15 Minutes 
8 Hours 
Medium 
(P3) 
15 Minutes 
24 Hours 
Low 
(P4) 
4 Business 
Hours 
3 Business Days 
NUIT aims to meet these targets 80% or more over a period of calendar month. 
** We expect to provide a 15-minute response time and 2-hour target resolution time for all 
incidents or requests received from deans, director and chairs, and their designated assistants 
with a 90% of higher compliance target. 
8.2 Service Request Response and Resolution Times 
Request Priority 
Response Time 
High 
1 Business Hour 
Resolution Time 
Medium, 
1 Business day 
4 Business Hours 
Low 
3 Business days 
8 Business Hours 
5 Business days 
NUIT aims to meet these targets 80% or more over a period of calendar month. 
8.4 Service Desk Metrics 
NUIT Service Desk function will aim to meet the following target conditions for key performance metrics. 
• 80% of calls answered or abandoned within 30 seconds (Service Level) Average 
• Speed of Answer: 30 seconds or less 
• Abandon Rate: 5-8% 
• First call resolution rate: 65% 
• Customer satisfaction rate: 85% 
8.6 Ongoing breach in service level 
In the event of an ongoing service level breach, NUIT will perform a root cause analysis and present 
options to review and determine appropriate steps to mitigate, which could include increasing 
staffing levels, process adjustments, or other remediation steps. 
9.0 Priority of service requests 
During periods of multiple simultaneous service requests, NUIT will allocate resources in 
accordance with the following criteria: 
1. High impact services (Mission Critical) 
2. High volume services 
3. Production services (opposed to training, quality assurance, test, development) 
4. Time sensitive services 
5. Other services 
10.0 Amendments or Termination 
This Agreement represents the entire agreement between the customer and NUIT and 
supersedes all prior negotiations or agreements, written or oral, which are not included 
herein. This Agreement may only be amended by written instructions executed by the 
customer and NUIT. 
Either party may end this agreement by providing 90 days written notice. All customer 
assets, in the form of data or otherwise, related to this agreement must be obtained by the 
customer before or on the date of termination. 
11.0 Support Outside of this Agreement 
Support under this agreement is recognized as a cooperative effort between customer, and 
Northwestern IT. The expertise Northwestern IT is providing is support for only the items 
outlined in this agreement. Should the customer determine there is a need for support for any 
items not specifically outlined in this agreement; the following procedure should be 
followed: 
A request should be made, in writing or in electronic form, to NUIT outlining the service 
requested. Note: requests for additional services should not be made to support personnel on 
site. 
Northwestern IT will create a proposal to deliver the service, detailing any new fees if 
incremental staff or services are required, and forward it to customer for review. Once 
approved by customer the proposal will be set forth as an addendum to this Agreement or as 
a new agreement if the requested services are foreign to the intentions of this Agreement. 
12.0 Billing Information 
Customer (Major Administration Unit):   
Fiscal Officer:    
Phone number:  
Email:    
Account:  
Sub Account:    
13.0 Cost of Agreement 
The below cost model only applies if there is not an existing Memorandum of 
Understanding. If a signed MOU exists please refer to that for the cost of agreement. 
Workstations: 
Mobile Devices 
Printers: 
$525/year 
$525/year 
No Charge (Limit one printer per workstation) 
Multi-function Devices: No Charge (Limit one multi-function device per workstation) 
Devices added mid-year prorated based upon applicable annual service charge for device.

"""

In [ ]:
from core.segmentation import segment_contract
clauses = await segment_contract(sample_contract_1)

print("Segmentation Output:")
print(clauses)

In [ ]:
clauses

## Final main.py Testing

In [ ]:
from dotenv import load_dotenv
load_dotenv()

from main import run_pipeline


In [ ]:
sample_contract_1="""

1.0 Service Level Agreement Overview 
This document is a Service Level Agreement between Northwestern IT (service provider) 
and the  
(customer), outlining the provisioning of IT services. 
The purpose of this agreement is to establish service expectations, identify provider and 
consumer responsibilities related to this service, specify how this service is supported, and 
clarify service charges. 
2.0 Business Service Description 
Northwestern IT (NUIT) agrees to provide Endpoint device management and support as a 
service to customer. 
NUIT will: provide 
• Device deployment – Procurement, Custom factory image loading including 
enterprise licensed common applications, asset tagging and labelling, pre-delivery 
and staging services, and system installation, system configuration and data 
migrations. Delivers and installs your fully integrated systems to customer 
designated user locations. 
• Secure endpoint management - pre-configuration of security solutions and 
installation of antivirus, data encryption and data backup solutions. Regular 
application of Operating system and application software patches. 
• Device Management - Agent monitoring to ensure proper tracking of device 
lifecycle 
• NUIT Service Desk - represents the entry point and first-touch support process for 
all hardware and software service-related requests. NUIT delivers a strategic, 
customized full-featured service desk that is driven by the goals of quality, value, 
and customer satisfaction. NUIT service desk is accountable to resolve, track, 
escalate and follow-up on all Endpoint systems related issues that affect customer 
productivity. 
• NUIT provides on-site Field Services support for endpoint systems, software, and 
peripherals support. We assist with installations, moves, adds and changes, as well 
as everyday hardware and peripheral support services. 
Authorized service delivery providers will be dispatched to your End User when 
hardware, software, or peripheral problems cannot be resolved remotely. 
• Disposition and redeployment– NUIT delivers efficient, secure and simple 
processes for data recoveries to disposals and refreshes. Ensures that system data 
is destroyed for security purposes. Disposal of Assets and manages Employee 
Purchase Programs. 
3.0 Agreement Period 
The period of this agreement is September 1, 2019 through August 30, 2020. Each year, 
this agreement will automatically extend for one calendar year unless it is modified or 
terminated as described in this document. 
4.0 Maintenance 
Maintenance of services provided in this agreement will be conducted in accordance with 
established NUIT Change Management practices, and will occur during scheduled service 
maintenance windows. Emergency maintenance, in response to service outages, may 
occur with no advanced notice to service consumers. Maintenance or outages of other, 
underlying or integrated, services may impact the services provided in this agreement. 
Notification of service maintenance and outages is available at: 
Service Status (https://service-status.northwestern.edu/). 
Information on how to subscribe to email service alerts can be found at: 
https://service-status.northwestern.edu/ 
5.0 Northwestern IT Responsibilities 
The following list specifies responsibilities of NUIT related to providing the service set 
forth in this agreement. Northwestern IT will be responsible for the following: 
Workstations (desktop and laptop computers) 
• Build workstations 
o Install one of NUIT approved computer images 
 Dual booting or machines with multiple operating systems will be 
assessed and approved by the Technology Support Services Director on 
a case by case basis based on business, research or teaching need. 
o Install additional software and or make additional updates to the endpoint 
device 
 Upon request 
 Subject to security approval 
o Include and manage 
 NUIT’ service automated software updating services (Kace/ JAMF) 
 Endpoint security software(Symantec Endpoint Protection) 
 Whole disk encryption (MBAM and FileVault) 
o Ensure desired work space has functional electrical and network access 
 Make recommendations for electrical or network access and advise 
CONDUITS and FM ordering processes. 
o Purchase, deliver and install workstation in desired work space 
o Troubleshooting wired or wireless network connectivity users, escalating to 
TNS has required. 
• Address user observed errors upon notification 
o Conduct troubleshooting and remediation efforts 
o Coordinate warranty work as required 
 Coordinate with Dell, Lenovo/CDWG for services (PCs) 
 Coordinate with Apple service technician (MACs) 
o Provide temporary loaner computer 
 Upon request 
 If operationally necessary 
 Subject to availability 
o Conference room and classroom support 
 Coordinate with Northwestern IT Audio/Visual team as need for 
support issue 
• Provide departmental fiscal office with semi-annual refresh notifications for 
workstations. 
• Assist with providing purchasing recommendations for new or replacement computers 
• Provide major OS updates as required for vendor support (e.g. Windows “Creator 
Updates”, MacOS 10.X point releases) 
• Provide disposition of end-of-life devices 
o Scrub data and remove hard drive 
o Send devices to Northwestern University Surplus and/or donate to Student 
Enrichment Services 
Mobile Devices 
The following support is offered upon request: 
• Make purchasing recommendations for Northwestern University owned mobile device 
• Configure mobile devise 
• Configure email 
• Coordinate activities with 3rd party vendors as required (University owned devices 
only) 
• Address user observed errors upon notification 
o Conduct troubleshooting and remediation efforts 
o Coordinate warranty work as required (University owned devices only) 
Printers 
The following support is offered upon request: 
• Set up printers 
• Organize for disposal of printers 
OU Administration (CampusAD) 
The following support is offered upon request: 
• Add new users to CampusAD 
• Create CampusAD accounts 
• Grant access to resources control via CampusAD 
• Delete users 
• Manage Group Policies 
Endpoint Management Services 
Management of the following endpoint management software’s: 
• Workstations will be patched no less than monthly for OS and application by use of 
management software, including Kace, JAMF 
• Full system backup of device including local machine stored data using Crashplan 
o Monitoring of backup services will be provided, with either tickets generated 
or user notification provided in the event that backups are not functioning 
Box 
The following support is offered upon request: 
• Install Box client software, such as Box Edit, Drive and Sync 
• Facilitate the creation of organization shares and group accounts 
Multi-function devices 
The following support is offered upon request: 
• Configure printing and scanning access to a multi-function device. 
• Address user observed errors upon notification 
o Conduct troubleshooting and remediation efforts 
o Coordinate warranty work as required 
Installation and Troubleshooting of customer specific software 
The following support is offered upon request: 
• Northwestern IT will install all customer specific software and will work with the 
user/experts in said software to ensure that the installation of these applications does not 
destabilize other desktop or application functionality 
• NUIT will work with the user/experts to contact the vendor of said software to 
accomplish same, but will not act independently of the user/experts in contacting 
vendors. 
Out of Scope 
The following services are explicitly not offered in this SLA. 
• Management of personal devices 
• Support for unmanaged devices such as servers, customer purchased devices 
New Technology 
Both Northwestern IT and the customer recognize that new technology adoption is 
important to the mission of the university. The customer may be introduced or considered 
for adoption during the course of this Agreement. At the mutual agreement of the NUIT 
and the customer, joint testing, demonstration, or evaluation of such technology may be 
undertaken. 
If the customer adopts a new computing technology and desires that NUIT support it, then 
the Customer Contact will consult with the TSS.  If they agree that the new technology 
has a limited effect on support time for the support personnel, then it will be incorporated 
into the current support activities. Otherwise, the Customer Contact will follow the 
procedure described in the section entitled “Support Outside of this Agreement”. 
Computer Support Policies 
Northwestern IT will support the University-owned computers of the customer based on 
the following policies: 
1. New purchases must meet the Northwestern IT recommended standards. New 
purchases will be set up with the NUIT software image. 
2. Existing computer hardware must meet the minimum hardware and operating system 
standards. 
3. Support level will be determined by the age of the machine 
Computer Age 
Support level 
Support description 
0-4 years 
Full Support 
Northwestern IT will ensure the working order of the 
computer, using reasonable staff time and support 
resources 
4-5 years 
Best Effort 
Northwestern IT will spend up to one hour fixing an 
issue. If the issue cannot be resolved at that time, NUIT 
will recommend replacing the computer and facilitate the 
recovery/transfer of data. 
5 years or 
older 
Out of Support 
Northwestern IT will facilitate the recovery/transfer of 
data, but will not attempt to resolve technical issues. 
Legacy Computers connected to instruments 
• Best effort support will be provided to computers connected to research equipment in 
consultation with Northwestern IT’s Information Security Office 
• Legacy equipment may need to be placed off network to avoid remote exploit or other 
risk 
6.0 Customer Responsibilities 
The following list contains the customer’s responsibilities related to their use of this 
service: 
General 
• Submit requests for services to the IT Support Center (by email, phone, or self-service 
ticket entry) 
• Submit notification of service interruptions or reductions to the IT Support Center 
• Provide funding for the purchase of any equipment covered by this SLA 
• Provide a list of the devices and their serial numbers for each device covered by this 
SLA: 
o Initial list will be provided from Kace export 
o Updates and changes to support environment will be managed in Kace, or its 
replacement for asset management 
• Notify NUIT of devices to be added to the SLA throughout the year through the new 
computer request form 
Workstations 
• Arrange to refresh workstations within four years of purchase 
• Provide funding for the refresh purchase 
Printers 
• Purchase a service warranty for each University owned printer that includes break-fix 
and full replacement 
7.0 Assumptions 
This service agreement assumes the following: 
• NUIT will bill Customer for support activity outside of the scope of this agreement at 
a standard service rate 
• Overtime work will be charged at 150% of the standard rate 
• After hours support work that requires support personal to return to work will include 
a minimum charge of three hours 
8.0 Requesting Support 
Support tickets, pertaining to service incidents (break-fix) and requests, should be 
submitted via the NUIT Service Desk and include the following information: 
• Customer contact information including: name, phone number, email address 
• Brief description of the service issue 
• Priority level of the incident or request 
The Service Desk staff will provide first level support to resolve the issue. Issues that are 
not resolved at first level will be escalated to Tier 2 support teams for resolution. 
Telephone calls received by the Service Desk outside of scheduled support hours will 
prompt callers to leave a message or page support staff for emergencies. The voice 
messages will be responded to on next business day. 
Customers are encouraged to check the NUIT Service Status web site 
https://service-status.northwestern.edu/ for scheduled maintenance events and service 
alerts. 
Contact Information 
Days 
Hours 
of 
operati
on 
Monday – 
Friday 
8:00 AM – 6:00 PM 
Service 
Desk 
Phone: (847) 491-4357 
(1-HELP) 
https://www.it.northwe 
stern.edu/supportcenter 
/index.html 
Saturday 
Not Available 
Sunday 
Summer Hours Monday – Friday 8:00 am – 5:00pm 
Not Available 
CLOSED: Winter/Spring Break, University holidays 
8.1 Incident Response and Resolution Times 
All incident tickets are assessed for customer impact and urgency to determine priority. A 
predefined Incident priority matrix is used as guidance in assessing incident priority. 
Priority 
Target 
Response 
Time 
Target 
Resolution 
Time 
Critical 
(P1) 
Immediate 
2 Hour 
High 
(P2) 
15 Minutes 
8 Hours 
Medium 
(P3) 
15 Minutes 
24 Hours 
Low 
(P4) 
4 Business 
Hours 
3 Business Days 
NUIT aims to meet these targets 80% or more over a period of calendar month. 
** We expect to provide a 15-minute response time and 2-hour target resolution time for all 
incidents or requests received from deans, director and chairs, and their designated assistants 
with a 90% of higher compliance target. 
8.2 Service Request Response and Resolution Times 
Request Priority 
Response Time 
High 
1 Business Hour 
Resolution Time 
Medium, 
1 Business day 
4 Business Hours 
Low 
3 Business days 
8 Business Hours 
5 Business days 
NUIT aims to meet these targets 80% or more over a period of calendar month. 
8.4 Service Desk Metrics 
NUIT Service Desk function will aim to meet the following target conditions for key performance metrics. 
• 80% of calls answered or abandoned within 30 seconds (Service Level) Average 
• Speed of Answer: 30 seconds or less 
• Abandon Rate: 5-8% 
• First call resolution rate: 65% 
• Customer satisfaction rate: 85% 
8.6 Ongoing breach in service level 
In the event of an ongoing service level breach, NUIT will perform a root cause analysis and present 
options to review and determine appropriate steps to mitigate, which could include increasing 
staffing levels, process adjustments, or other remediation steps. 
9.0 Priority of service requests 
During periods of multiple simultaneous service requests, NUIT will allocate resources in 
accordance with the following criteria: 
1. High impact services (Mission Critical) 
2. High volume services 
3. Production services (opposed to training, quality assurance, test, development) 
4. Time sensitive services 
5. Other services 
10.0 Amendments or Termination 
This Agreement represents the entire agreement between the customer and NUIT and 
supersedes all prior negotiations or agreements, written or oral, which are not included 
herein. This Agreement may only be amended by written instructions executed by the 
customer and NUIT. 
Either party may end this agreement by providing 90 days written notice. All customer 
assets, in the form of data or otherwise, related to this agreement must be obtained by the 
customer before or on the date of termination. 
11.0 Support Outside of this Agreement 
Support under this agreement is recognized as a cooperative effort between customer, and 
Northwestern IT. The expertise Northwestern IT is providing is support for only the items 
outlined in this agreement. Should the customer determine there is a need for support for any 
items not specifically outlined in this agreement; the following procedure should be 
followed: 
A request should be made, in writing or in electronic form, to NUIT outlining the service 
requested. Note: requests for additional services should not be made to support personnel on 
site. 
Northwestern IT will create a proposal to deliver the service, detailing any new fees if 
incremental staff or services are required, and forward it to customer for review. Once 
approved by customer the proposal will be set forth as an addendum to this Agreement or as 
a new agreement if the requested services are foreign to the intentions of this Agreement. 
12.0 Billing Information 
Customer (Major Administration Unit):   
Fiscal Officer:    
Phone number:  
Email:    
Account:  
Sub Account:    
13.0 Cost of Agreement 
The below cost model only applies if there is not an existing Memorandum of 
Understanding. If a signed MOU exists please refer to that for the cost of agreement. 
Workstations: 
Mobile Devices 
Printers: 
$525/year 
$525/year 
No Charge (Limit one printer per workstation) 
Multi-function Devices: No Charge (Limit one multi-function device per workstation) 
Devices added mid-year prorated based upon applicable annual service charge for device.

"""

In [ ]:
# contract_text = """
# Client shall pay all undisputed invoices within thirty (30) days of receipt.
# Late payments shall incur interest at 2% per month.

# Vendor shall indemnify, defend, and hold harmless Client against any and all third-party claims arising from Vendor’s breach of this Agreement.

# The parties agree to cooperate in good faith to resolve disputes amicably.
# """


In [ ]:
results = await run_pipeline(sample_contract_1)

In [ ]:
import json
print(json.dumps(results, indent=2))